# Extract the dataset

In [2]:
import zipfile
import os
import shutil

def extract_images(path):
    for root, subdir, files in os.walk(img_path):
        for f in files:
            if '.jpg' in f:
                return False
    return True


base_path = os.getcwd()
img_path = f"{base_path}/data/intel_image_classification"

if extract_images(img_path):
    with zipfile.ZipFile(f"{base_path}/data/archive.zip", 'r') as zip_ref:
        zip_ref.extractall(img_path)
        # We are only interested on model performance, so no prediction data is used
        shutil.rmtree(f"{img_path}/seg_pred")

# Load images

In [3]:
from tensorflow import keras
import numpy as np
from sklearn import preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tqdm import tqdm


def prepare_targets(y_train, y_test):
	le = preprocessing.LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

def import_data(path, keras_encoder):
    x_train_list, y_train_list, x_test_list, y_test_list = [], [], [], []

    for root, subdir, files in os.walk(img_path):
            if len(files) < 2:
                continue
            for f in tqdm(files):
                if f.endswith('.jpg'):
                    file_path = os.path.normpath(os.path.join(root, f))
                    label = file_path.split(os.sep)[len(file_path.split(os.sep)) - 2]
                    x = np.expand_dims(
                        image.img_to_array(
                            image.load_img(
                                file_path, 
                                target_size=(224, 224)
                                )
                            ), 
                        axis=0
                        )
                    x = preprocess_input(x)
                    x = model.predict(x)
                    if 'seg_train' in file_path:
                        x_train_list.append(x)
                        y_train_list.append(label)
                    else:
                        x_test_list.append(x)
                        y_test_list.append(label)
                    
                    # Free memory
                    x = None

    y_train, y_test = prepare_targets(
        np.array(y_train_list),
        np.array(y_test_list)
        )

    return (np.vstack(x_train_list), y_train), (np.vstack(x_test_list), y_test)

In [21]:
model = VGG16(weights='imagenet', include_top=False)
(x_train, y_train), (x_test, y_test) = import_data(img_path, model)

100%|██████████| 437/437 [01:10<00:00,  6.22it/s]


In [29]:
# Save the numpy array for future training
from numpy import asarray
from numpy import savez_compressed

save_path = f"{base_path}/fe_data/include_top_false"

savez_compressed(f"{save_path}/x_train.npz", x_train)
savez_compressed(f"{save_path}/y_train.npz", y_train)
savez_compressed(f"{save_path}/x_test.npz", x_test)
savez_compressed(f"{save_path}/y_test.npz", y_test)

In [31]:
# Free up numpy array memory for further feature extration with different VGG16 cuts
x_train, y_train, x_test, y_test = None, None, None, None

In [4]:
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet')

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 64373286793e3c8b2b4e3219cbf3544b so we will re-download the data.
553467904/553467096 [==============================] - 361s 1us/step


In [5]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
(x_train, y_train), (x_test, y_test) = import_data(img_path, model)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      